In [2]:
from env import googleapi, openapi, hfapi

In [3]:
"""from huggingface_hub import login
login(token=hfapi)  # Only required once per environment"""

C:\Users\91962\.conda\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/whoami-v2 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020D28903A60>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 66156de4-5245-4eb6-a864-f8263d1adcb1)')

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

In [ ]:
print("hello {hehe}".format(hehe=5))

In [ ]:
template1 = """
You are a consultant writing formal documents. Generate a **Terms of Reference (ToR)** for the following project.

**Project Description:**  
{description}

Your sole task is to generate a professionally formatted ToR document using the structure below.  
**Do not include disclaimers, warnings, or comments** about AI generation, authorship, or the draft nature of the document.  
**Do not leave placeholders for dates, durations, or days** — your only responsibility is to create the Terms of Reference content.

**ToR Structure:**

1. **Background**  
   - Provide relevant context and justification for the project.

2. **Objectives**  
   - Clearly state the specific goals of the project.

3. **Scope of Work**  
   - Describe the tasks and activities to be undertaken.  
   - Define deliverables and timelines.

4. **Methodology**  
   - Outline the approach or methods to be used to complete the work.

5. **Deliverables**  
   - List the specific outputs expected from the assignment.

6. **Duration**  
   - Mention the timeline and key milestones.

7. **Qualifications and Experience**  
   - Specify the skills, background, or expertise required.

8. **Reporting Requirements**  
   - Identify who the consultant will report to and how frequently.

9. **Reference Documents**  
   - List any source materials, policies, or frameworks to be consulted.

**Return only the completed Terms of Reference** in a formal, professional tone suitable for official use.
"""
prompt1 = PromptTemplate.from_template(template)

In [ ]:
template = """
You are a consultant specializing in drafting formal Terms of Reference (ToR) documents. Based on the project description provided, generate a comprehensive ToR following the structure below:

1. **Background**: Provide context and rationale for the project.
2. **Objectives**: Outline the goals the project aims to achieve.
3. **Scope of Work**: Detail the tasks and activities to be undertaken.
4. **Deliverables**: Specify the expected outputs and outcomes.
5. **Methodology**: Describe the approach and techniques to be used.
6. **Expertise Required**: List the qualifications and experience needed.
7. **Reporting Requirements**: Define how progress and results will be communicated.
8. **Work Plan and Timeline**: Present a schedule of activities and milestones.
9. **Budget and Resources**: Outline the financial and material resources allocated.
10. **Risks and Mitigation**: Identify potential challenges and strategies to address them.
11. **Approval and Sign-Off**: Specify the authorities responsible for approving the ToR.

Project Description:
{description}

Please generate the ToR accordingly.
"""

prompt2 = PromptTemplate.from_template(template)

In [ ]:
# get the dataset convert into pandas df
from datasets import load_dataset
ds = load_dataset("arjunssat/rfp_train_v1")
df = ds["train"].to_pandas()

In [ ]:
df.head()

In [ ]:
df.shape

### only body column is meaningful for us
### this is how it looks like

In [ ]:
df["body"][0][1:-1] # bypass square bracket at start and end

In [ ]:
## extract taht info in another variable
info = [df["body"][x][1:-1] for x in range(len(df))]

In [ ]:
print(prompt1)

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = googleapi

### testing inference

In [ ]:
from markdownify import markdownify as md

def unmarkdown(text):
    return md(text)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
import ollama
models = ["llama3.2", "gemma3:4b", "gemini2.5", "qwen3:4b",  "deepseek-r1:7b"]
responses = []
m, o = 0, 0
while o < len(df):
    # Loop through the models reset when crossing line
    if m >= 5:
        m = 0
    model = models[m]
    print("on index", o, model)
    
    #print("_"*100)
    #print(f"Using model: {model}")
    if model == "gemini"
    if model == "gemini2.5":
        # Instantiate the Gemini model (e.g., gemini-pro or gemini-2.0-flash-001)
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17", temperature=0.7)
        # Create an LLMChain with the LLM and prompt
        chain = LLMChain(llm=llm, prompt=prompt1)
        # Run the chain with your input
        response = chain.run(description=info[o])
        response = unmarkdown(response)
        #print(response)
        responses.append([model, response])
    else:
        # Send a request to the LLM
        response = ollama.chat(model=model, messages=[{"role": "user", "content": template1.format(description = info[o])}])
        #responses.append([model, response])
        if model in ["qwen3:4b",  "deepseek-r1:7b"]:
            response = response["message"]["content"].split("</think>")[1]
        else:
            response = response["message"]["content"]
        response = unmarkdown(response)
        #print(response)
        
        #print(response)
        responses.append([model, response])
        
    o +=1
    m +=1

In [ ]:
"""import os
os.environ["OPENAI_API_KEY"] = openapi"""